In [36]:
import pandas as pd
import numpy as np
#import graphlab as tc
import turicreate as tc
from tqdm.notebook import tqdm
import turicreate.aggregate as agg

In [27]:
def read_labels():
    apk_label_ds1_path = '../data/labels_vt_mal_2017_2020_az_2020_benign_hashed.csv'
    ds1_labels_df = pd.read_csv(apk_label_ds1_path)
    ds1_labels_df.set_index(ds1_labels_df.apk.apply(lambda x:x.upper()), inplace=True)
    ds1_labels_df.drop(labels='apk',axis=1, inplace=True)
#    ds1_labels_df.head(2)
    print("Got data: ", len(ds1_labels_df))
    return ds1_labels_df

In [76]:
def transform_app_data(fname='../data/sample_10000_vt_mal_2017_2020_az_2020_benign_hashed_md5.csv'):
    sf_ds1_full = gl.SFrame.read_csv(fname, header=False, verbose=False)
    _=sf_ds1_full.rename({'X3':'fcount'})
    sf_ds1_full['apk']=sf_ds1_full['X1'].apply(lambda x: x.upper())
    sf_ds1_full['function'] = sf_ds1_full['X2'].apply(lambda x: x.upper())
    
    _=sf_ds1_full.remove_columns(['X1', 'X2'])
    
    print("Unique APK:", len(set(sf_ds1_full['apk'].unique())))
    print("Unique functions:", len(set(sf_ds1_full['function'].unique())))

    outname = fname.replace('data', 'binarydata').replace('.csv', '.sframe')
    sf_ds1_full.save(outname, format='binary')

transform_app_data()

('Unique APK:', 10003)
('Unique functions:', 699854)


In [2]:
mw = tc.load_sframe('../binarydata/sample_10000_vt_mal_2017_2020_az_2020_benign_hashed_md5.sframe')

In [3]:
mw.head()

fcount,apk,function
1,3407F2E5AB21E8966BDFAF33D805DE7AC97BED3A7454C1 ...,2D6877D28225C6E0F115275774CDA94C ...
1,006373DF53793664DDD73C7A96B18794A10DE6FAAD185B ...,4B673494945E9BF470675331FC1D6B27 ...
2,238230D0B795F97E596FA7379AF0769D74ED3810976092 ...,96D7A6B3A2BEA95CA0E3B74FE3C4E7F8 ...
1,25A3382B3FAAAC9D5A14FE8DE914D47B3BC6284599FD27 ...,A6D7D5D7968AE7BDC62AD696AE7E6BD1 ...
1,4D0F5074E2A7579DBC87C0D8483EFEB3D31B24E0008BD1 ...,20E0EB29F9F144A01276B87CFC0AC2A1 ...
1,94F2FB12A21085764C85DB0E9E4B381EEDD1DC65E4F2D7 ...,F66DB25FFCDEEB2D40CE702261BEEE83 ...
1,9AC8EDE1BF55180F0F90A8B127ACDF21AE9A8916FBE0E6 ...,B9AFD08C24947FE6E37A82672795427B ...
1,005CDFD24DEF5924C91E68DC0CF18D18B91D6118D4BE51 ...,4ABCBF64B57FAD928D0414890D41CC95 ...
1,0461C5356AF1B678D91A874041AAF9978764BC8A3E1833 ...,7D72BE96A185F93BD36B924811C8EEC5 ...
1,4A69FD5D71F3010B08B2C2E51CF942D2513D48ADD872C9 ...,4ABCBF64B57FAD928D0414890D41CC95 ...


In [9]:
benign_ds1 = ds1_labels_df[ds1_labels_df['malware_label'] == False]
print(len(benign_ds1))
malware_ds1 = ds1_labels_df[ds1_labels_df['malware_label'] == True]
print(len(malware_ds1))

9389
18329


## Gamma network 


In [4]:
mw = tc.load_sframe('../binarydata/sample_10000_vt_mal_2017_2020_az_2020_benign_hashed_md5.sframe')

In [4]:
def get_sample(mw, frac):
    apks = mw['apk'].unique()
    sample_apks = apks.sample(fraction=frac, seed=42)
    return mw.filter_by(sample_apks, column_name='apk')
    

In [6]:
subsamp = get_sample(mw=mw, frac=0.1)

In [7]:
len(subsamp['apk'].unique())

993

### Right position for $k$

it is not sufficient to set k for `get_similar_items`. The recomended performs a cut-off anyways. The $k$ has to be set in the `create` method (`only_top_k`). 

There is also a `threshold` in `create` initialization, but apparently this has less influece.

In [8]:
k = len(subsamp['apk'].unique())


sim_recom = tc.item_similarity_recommender.create(subsamp, 
                                                  user_id='function',
                                                  item_id='apk',
                                                  similarity_type='jaccard',
                                                  verbose=False, only_top_k=k)

In [9]:
def get_funcs(tab, apk):
    return tab[tab['apk']==apk]['function']

def alt_jaccard(apk1, apk2, tab):
    ap1func = set(get_funcs(tab, apk1))
    ap2func = set(get_funcs(tab, apk2))
    
    return float(len(ap1func & ap2func)) / float(len(ap1func| ap2func))

In [18]:
def get_similar_apks(apk):
    similar_items = sim_recom.get_similar_items([apk], k=k)['similar', 'score']
    similar_items.materialize()
    return similar_items

def get_jaccard_sim(apk1, apk2):
    similar_to_apk1 = get_similar_apks(apk1)
    try:
        return similar_to_apk1[similar_to_apk1['similar']==apk2]['score'][0]
    except:
        return 0

def get_recommender(data):
    k = len(data['apk'].unique())
    return k, tc.item_similarity_recommender.create(data, 
                                                 user_id='function',
                                                 item_id='apk',
                                                 similarity_type='jaccard',
                                                 verbose=False, only_top_k=k)
    
def jaccard_dist(apk1, apk2):
    return 1 - get_jaccard_sim(apk1, apk2)

def aio_distance(apk1, apk2, k, recommender):
    similar_items = recommender.get_similar_items([apk1], k=k)['similar', 'score']
    similar_items.materialize()
    
    try:
        return 1-similar_items[similar_items['similar']==apk2]['score'][0]
    except:
        return 1
    

### Precision issues

In [14]:
k, rec = get_recommender(subsamp)

In [15]:
sample_apks = subsamp['apk'].unique()

In [16]:
apk1 = sample_apks[0]
apk2 = sample_apks[1]
print(apk1)

2D151F82FA37F7E52342909747348C7613905B6CDB82CB8005A38299F0D908EB


In [17]:
get_jaccard_sim(apk1, apk2), alt_jaccard(apk1, apk2, subsamp)#aio_distance(apk1, apk2, rec)

(0.00043523311614990234, 0.0004352557127312296)

In [18]:
ap1func = set(get_funcs(subsamp, apk1))
ap2func = set(get_funcs(subsamp, apk2))

len(ap1func & ap2func), len(ap1func | ap2func)

(2, 4595)

In [23]:

cup, cut = Out[18]
cup/ cut

0.0004352557127312296

In [229]:
%%timeit
for apks in sample_apks[1:100]:
    dist = get_jaccard_sim(apk1=apk1, apk2=apks)

1 loop, best of 3: 603 ms per loop


### Pandas seems to much faster for labels

In [28]:
labels = read_labels()

Got data:  27718


In [29]:
def classifier(apk):
    return [[0, 1], [1, 0]][int(labels.loc[apk]['malware_label'])]

In [30]:
apk_label_ds1_path = '../data/labels_vt_mal_2017_2020_az_2020_benign_hashed.csv'
ds1_labels = tc.SFrame.read_csv(apk_label_ds1_path, header=True, verbose=False)
ds1_labels['mwl'] = ds1_labels['malware_label'].apply(lambda x: int(x=='TRUE'))
ds1_labels['apk'] = ds1_labels['apk'].apply(lambda x: x.upper())
_=ds1_labels.remove_columns(['malware_label'])

In [31]:
def alt_classifier(apk):
    return [[0, 1], [1, 0]][ds1_labels[ds1_labels['apk']==apk]['mwl'][0]]

In [36]:
%%timeit
# classify using turi
for apk1 in sample_apks:
    r = alt_classifier(apk1)

44.2 s ± 5.91 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit
# the power of pandas
for apk1 in sample_apks:
    r = classifier(apk1)

73.5 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:

def create_voting_net_alt(gamma, apns, classifier, distance):
    net = dict()
    for a in apns:
        insert = True
        for n in net.keys():
            if distance(a, n) <= gamma:
                insert = False
                net[n] = list(np.add(net[n], classifier(a)))
                break
        if insert:
            net[a] = classifier(a)
    return net

In [23]:
net = create_voting_net_alt(gamma=0.6, apns=sample_apks, classifier=classifier, distance=jaccard_dist)

In [39]:
k, rec = get_recommender(subsamp)
distance = lambda x,y: aio_distance(x,y, k, rec)
net2 = create_voting_net_alt(gamma=0.6, apns=sample_apks, classifier=classifier, distance=distance)

In [41]:
apks = [a for a in sample_apks]

In [42]:
create_voting_net_alt(gamma=0.6, apns=apks, classifier=classifier, 
                      distance=)

['...']

In [52]:
df = subsamp.to_dataframe()

df.drop(columns=['fcount'], axis=1, inplace=True)


funcs_smp = df.groupby(by='apk')['function'].apply(set)

In [75]:
def jaccard(apid1, apid2, funcs):
    p1 = funcs[apid1]
    p2 = funcs[apid2]

    return 1 - len(p1 & p2)/len(p1|p2)

def panda_jaccard(apid1, apid2, funcs):
    p1 = funcs.loc[apid1]
    p2 = funcs.loc[apid2]
    return 1 - len(p1 & p2)/len(p1|p2)

In [57]:
apk1 = funcs_smp.index[0]
apk2 = funcs_smp.index[1]

In [58]:
ffn = funcs_smp.to_numpy()

In [72]:
pairs = [(np.random.randint(len(funcs_smp.index)), np.random.randint(len(funcs_smp.index))) for _ in range(1000)]
apk_pairs = [(funcs_smp.index[a], funcs_smp.index[b]) for a,b in pairs]

In [80]:
%%timeit
dis = [jaccard(*a, ffn) for a in pairs]

624 ms ± 6.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [81]:
%%timeit
disa = [distance(*b) for b in apk_pairs]

3.12 s ± 85.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
panda_distance = lambda x,y: panda_jaccard(x,y, funcs_smp)

In [82]:
%%timeit
disp = [panda_distance(*a) for a in apk_pairs]

674 ms ± 6.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Checking merging

In [10]:
bigsam = get_sample(mw=mw, frac=0.1)

In [30]:
import numpy as np

def get_part_indexes(dfs, num_parts, size, seed=42):
    permuted_indices = np.random.permutation(len(dfs))
    return [permuted_indices[i:i+size] for i in range(0, size*num_parts, size)]

In [62]:
p1 = get_part_indexes(bigsam['apk'].unique(), num_parts=2, size=100)

## Notices
1. convert hashes to ints?
2. subsample


In [82]:
apks = bigsam['apk'].unique()

In [106]:
def get_sarray_parts(sa, num_parts, size):
    permuted_indices = np.random.permutation(len(sa))
    return [[sa[permuted_indices[j]] for j in range(i,i+size)] for i in range(0, size*num_parts, size)]

In [107]:
parts = get_sarray_parts(apks, 5, 100)

In [108]:
len(parts), len(parts[4])

(5, 100)

In [119]:
k,rec = get_recommender(bigsam)
distance = lambda x,y: aio_distance(x,y,k, rec)

Recsys training: model = item_similarity

In [121]:
nets = [create_voting_net_alt(gamma=0.6, apns=part, classifier=classifier, distance=distance) for part in parts]

In [128]:
sum(map(sum,nets[4].values()))

100

In [129]:
for n in nets:
    print(len(n.keys()))

54
49
49
52
58


In [132]:
def create_aggregating_net(gamma, apns, distance):
    net = dict()

    for a in apns:
        insert = True
        for n in net.keys():
            if distance(a, n) <= gamma:
                insert = False
                net[n].append(a)
                break  
        if insert:
            net[a] = list()

    return net

def merge_voting_nets(nets, distance, gamma):
    apns = []
    for net in nets:
        apns += list(net.keys())

    nn = create_aggregating_net(gamma=gamma,
                                apns=apns,
                                distance=distance)
    
    # transfer the "votes" from original networks to just created new anchors
    targ = dict()
    for k, v in nn.items():
        for net in nets:
            if k in net:
                targ[k] = net.get(k)
                break

        for el in v:
            for net in nets:
                if el in net:
                    targ[k] = list(np.add(targ[k], net[el]))
                    break

    return targ

In [133]:
mr = merge_voting_nets(nets=nets, distance=distance, gamma=0.6)

In [137]:
len(mr.keys()), sum(map(lambda x: len(x.keys()), nets))

(162, 262)

## Network creation straight from recommender

In [2]:
mw = tc.load_sframe('../binarydata/sample_10000_vt_mal_2017_2020_az_2020_benign_hashed_md5.sframe')

In [5]:
subsamp = get_sample(mw=mw, frac=0.1)

In [6]:
apks = subsamp['apk'].unique()

In [7]:
k = len(apks)

In [8]:
sim_recom = tc.item_similarity_recommender.create(subsamp, 
                                                  user_id='function',
                                                  item_id='apk',
                                                  similarity_type='jaccard',
                                                  verbose=False,only_top_k=k) #threshold=0.5

In [22]:
itms = sim_recom.get_similar_items(apks, k=k)

In [73]:
#itms.materialize()

In [24]:
ngw=itms[itms['score']<1-0.3] 

In [27]:
appk = ngw['apk'][0]

In [47]:
gw = ngw.groupby(key_column_names='apk', operations={'sims': agg.COUNT_DISTINCT('similar')})

In [51]:
gw[gw['sims']<10]

apk,sims


In [52]:
gw2=itms[itms['score']>=1-0.7] 

In [53]:
gwc = gw2.groupby(key_column_names='apk', operations={'sims': agg.COUNT_DISTINCT('similar')})

In [60]:
npk = gwc[gwc['sims']<2][0]['apk']

apk,sims
62DC51A642A9FBB733E5A987B7E44757119BF4E2653B5B ...,1


In [15]:
def create_network(data, gamma):
    apks = data['apk'].unique()
    k = len(apks)
    sim_recom = tc.item_similarity_recommender.create(data, 
                                                  user_id='function',
                                                  item_id='apk',
                                                  similarity_type='jaccard',
                                                  verbose=False,only_top_k=k) #threshold=0.5
    itms = sim_recom.get_similar_items(apks, k=k)
#    itms = itms.remove_column('rank')
    gw=itms[itms['score']>=1-gamma]

    net = dict()
    napks = set(apks)
    while len(napks) > 0:
        apk = napks.pop()
        net[apk] = set(gw[gw['apk']==apk]['similar'])
        napks = napks - net[apk]
        
    return net

In [80]:
neti = create_network(data=subsamp, gamma=0.7)

In [81]:
len(neti)

199

In [101]:
itms.head(1)

apk,similar,score
2D151F82FA37F7E52342909747348C7613905B6CDB82CB ...,A12B8CE4DE8D33ED120BB48ABC118FADEE3A16FF9F4C6B ...,0.6900672316551208


In [178]:
def setup_rec(data):
    apks = data['apk'].unique()
    k = len(apks)
    sim_recom = tc.item_similarity_recommender.create(data, 
                                                      user_id='function',
                                                      item_id='apk',
                                                      similarity_type='jaccard',
                                                      verbose=False,only_top_k=k)
    return apks, k, sim_recom

In [181]:
def create_network_alt(data, gamma, apks, k, sim_recom):
    itms = sim_recom.get_similar_items(apks, k=k)

    # potentially loosing some anchors here? 
    gw=itms[itms['score']>=1-gamma] 
    gw = gw.groupby(key_column_names='apk', operations={'sims': agg.DISTINCT('similar')})
    ws = set(gw['apk'])

    net = dict()
    already_added = set()
    while len(ws)>0:
        w= ws.pop()

        simp = set(gw[gw['apk']==w]['sims'][0]) 
        # alt:        gw[(gw['apk']==w)&(gw['score']>=1-gamma)]
        simp = simp - already_added

        net[w] = simp
        already_added.update(simp)
        already_added.add(w)

        ws = ws - simp
    return net

In [179]:
apks, k, sim_recom = setup_rec(subsamp)

In [182]:
nn = create_network_alt(data=subsamp, gamma=.7, apks=apks, k=k, sim_recom=sim_recom)

In [183]:
len(nn), sum(map(len, nn.values())), len(nn)+ sum(map(len, nn.values()))

(107, 794, 901)

In [16]:
nb = create_network(data=subsamp, gamma=.7)

In [35]:
def check_net(net, distance, gamma):
    ml = list(net.keys())
    for i, a in tqdm(enumerate(ml)):
        for b in ml[i:]:
            if a==b:
                continue

            d = distance(a, b)
            if d < gamma:
                print(f"d({a}, {b}) =={d}")

In [171]:
check_net(net=nn, distance=lambda x,y: aio_distance(apk1=x, apk2=y, k=k, recommender=sim_recom), gamma=.7)

In [172]:
def check_kys(net, distance, gamma):
    for a, k in tqdm(net.items()):
        for wr in k:
            if distance(a, wr)> gamma:
                print(f"|{a}, {wr}| = {distance(a,wr)}")

In [173]:
check_kys(net=nn, distance=lambda x,y: aio_distance(x,y, k=k, recommender=sim_recom), gamma=.7)

In [175]:
sum(map(len, nn.values())), len(nn), sum(map(len, nn.values())) + len(nn)

(794, 107, 901)

In [176]:
k

993

In [ ]:
# Issues
-> solitary nodes (no similarities at all?)


In [9]:
nns = {1: [2, 3], 2: [1, 3, 4], 3: [2, 4], 4: [2, 3]}

In [20]:
net = dict()
already_added = set()
for a, b in nns.items():
    if a in already_added:
        continue
    net[a] = set(b) - already_added
    already_added.update(set(b))
    already_added.add(a)
    

In [21]:
net

{1: {2, 3}, 4: set()}

## artificial data

In [167]:
def f_create_network(data, gamma):
    apks = data['apk'].unique()
    k = apks.shape[0]
    sim_recom = tc.item_similarity_recommender.create(data, 
                                                      user_id='function', 
                                                      item_id='apk', 
                                                      similarity_type='jaccard', 
                                                      only_top_k=k, verbose=False)
    itms = sim_recom.get_similar_items(art['apk'].unique(), k=k)
    # 2. missing more "distant nodes", "not aggregating nodes"
    gw=itms[itms['score']>=1-gamma].groupby(key_column_names='apk', operations={'sims': agg.DISTINCT('similar')})
    
    ws = set(gw['apk'])
    net = dict()
    already_added = set()
    while len(ws)>0:
        w= ws.pop()

        simp = set(gw[gw['apk']==w]['sims'][0])
        simp = simp - already_added

        net[w] = list(simp)
        already_added.update(simp)
        already_added.add(w)

        ws = ws - simp
    
        
    # add solitary nodes & not-aggregating nodes
    if len(already_added)> 0:
        nds = apks.filter_by(list(already_added), exclude=True)
    else:
        nds = apks
        
    for n in nds:
        net[n] = []
        
    return net
    
    

In [168]:
cons = {'A': [1, 2, 3], 'B': [2, 5, 6], 'C': [1, 4, 3], 'D': [6, 5], 'E': [1, 2], 'F': [8], 'G': [9]}

In [169]:
dat2 = {'apk': [], 'function': []}
for apk, lsts in cons.items():
    for fncs in lsts:
        dat2['apk'].append(apk)
        dat2['function'].append(fncs)

In [170]:
art = tc.SFrame(data=dat2)

In [175]:
f_create_network(data=art, gamma=.7)

{'A': ['E', 'C'], 'B': ['D'], 'F': [], 'G': []}